<a href="https://colab.research.google.com/github/AngelTroncoso/Agentes_Gratis/blob/main/Carga_pdf_doc_csv%2Ctex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

LangChain es una biblioteca de Python que facilita la creación de aplicaciones de IA basadas en lenguaje. Una de sus funcionalidades clave es la capacidad de cargar y procesar diversos tipos de documentos, incluyendo PDF y CSV.

¿Por qué usar LangChain para cargar PDF y CSV?
Unificación de formatos: Permite tratar archivos PDF y CSV de manera similar, facilitando la creación de modelos y aplicaciones que trabajen con múltiples fuentes de datos.
Preprocesamiento: Ofrece herramientas para limpiar y estructurar los datos extraídos de los documentos, preparándolos para su uso en modelos de lenguaje.
Integración con LLMs: Se conecta fácilmente con grandes modelos de lenguaje (LLMs) para permitir la generación de texto, traducción, resumen y mucho más, a partir de los datos cargados.
Carga de archivos CSV
Los archivos CSV (Comma-Separated Values) son fáciles de cargar con LangChain. Cada fila del CSV se convierte en un documento individual.

Python

from langchain.document_loaders import CSVLoader​loader = CSVLoader(file_path="my_data.csv") documents = loader.load()

Personalización:

Columnas específicas: Puedes seleccionar solo las columnas que te interesan.
Separadores: Si tu CSV utiliza un separador diferente a la coma, puedes especificarlo.
Tipos de datos: Puedes convertir las columnas a tipos de datos específicos (por ejemplo, fechas, números).
Carga de archivos PDF
Cargar archivos PDF con LangChain es un poco más complejo debido a la estructura variada de estos documentos. LangChain ofrece varias opciones:

PDFMiner: Una biblioteca de Python para extraer texto de archivos PDF.
PyMuPDF: Otra biblioteca popular para trabajar con PDF.
LayoutParser: Una biblioteca más avanzada que puede extraer texto, tablas e imágenes de PDF con mayor precisión.
Python

from langchain.document_loaders import PyPDFLoader​loader = PyPDFLoader("my_document.pdf") pages = loader.load()

Consideraciones:

Complejidad del PDF: PDFs con tablas, imágenes o formatos complejos pueden requerir un procesamiento más elaborado.
OCR: Si tu PDF contiene texto escaneado, necesitarás utilizar herramientas de reconocimiento óptico de caracteres (OCR).
Uso posterior de los documentos cargados
Una vez cargados los documentos, puedes utilizarlos para:

Crear un índice: Para realizar búsquedas rápidas dentro de los documentos.
Entrenar un modelo de lenguaje: Para crear un modelo que pueda responder preguntas sobre los datos.
Generar texto: Para crear resúmenes, traducciones o nuevas piezas de contenido basadas en los documentos.
Python

from langchain.vectorstores import FAISS​# Crear un índice vectorstore = FAISS.from_documents(documents)​# Realizar una búsqueda query = "Cuál es la capital de Francia?" docs = vectorstore.similarity_search(query)

Ejemplos de uso
Chatbots: Crear un chatbot que pueda responder preguntas sobre un conjunto de documentos PDF y CSV.
Sistemas de recomendación: Recomendar productos o servicios basados en las preferencias de un usuario almacenadas en un CSV.
Resumen de documentos: Generar resúmenes concisos de largos documentos PDF.
Análisis de sentimientos: Analizar la opinión de los clientes a partir de comentarios almacenados en CSV.
Conclusión
LangChain simplifica significativamente el proceso de cargar y procesar documentos PDF y CSV. Al combinar la flexibilidad de LangChain con la potencia de los grandes modelos de lenguaje, puedes crear aplicaciones de IA sofisticadas y personalizadas.

# Chat Memory

In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

In [ ]:
!pip install langchain_openai langchain

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

model = ChatOpenAI(model="gpt-3.5-turbo")

In [ ]:
model.invoke([HumanMessage(content='Hi! my name is bob')]).content

In [ ]:
model.invoke([HumanMessage(content='What is my name?')]).content

In [ ]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hello Bob! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
).content

In [ ]:
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
from langchain_core.runnables.history import RunnableWithMessageHistory

In [ ]:
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
  if session_id not in store:
    store[session_id] = InMemoryChatMessageHistory()
  return store[session_id]

In [ ]:
with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [ ]:
config = {'configurable': {'session_id': 'abc2'}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi! I am Bob')],
     config=config
)

In [ ]:
response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is my name?')],
     config=config
)

In [ ]:
response.content

In [ ]:
store

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='How are you?')],
     config=config
)

In [ ]:
store

In [ ]:
config = {'configurable': {'session_id': 'abc3'}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is my name?')],
     config=config
)
response

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi! I am Carli')],
     config=config
)
response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is my name?')],
     config=config
)
response

In [ ]:
store

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            'system', 'You are a helpful assistant. Answer all question to the best of your abity.'
        ),
        MessagesPlaceholder(variable_name='messages')

    ]
)

In [ ]:
chain  = prompt | model

In [ ]:
response = chain.invoke(
    {
        'messages': [HumanMessage(content='Hi! I am Bob')]
    }
)
response.content

In [ ]:
response = chain.invoke(
    {
        'messages': [HumanMessage(content='What is your name?')]
    }
)
response.content

In [ ]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [ ]:
config = { 'configurable': {'session_id': 'abc5'}}

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='What is your name?')],
    config=config
)
response.content

In [ ]:
response = with_message_history.invoke(
    [HumanMessage(content='Hi! I am Bob')],
    config=config
)
response.content

Resumen
1. Document Loaders: Cargar Archivos en LangChain
Los Document Loaders son herramientas especializadas en LangChain que permiten cargar información desde varios formatos de archivos, como PDF, CSV, Markdown, HTML, entre otros. Cada tipo de archivo tiene su propio loader específico que facilita la extracción de texto y metadatos.

a) Carga de PDFs  
Uno de los ejemplos más comunes es cargar y trabajar con PDFs. Para ello, utilizamos el paquete PyPDF dentro de LangChain.  

PyPDF es un cargador de PDFs que te permite acceder al contenido y los metadatos del archivo, como el número de páginas, los títulos o cualquier otra información relevante.  
Pasos para cargar un PDF:  

Instalar el paquete PyPDF y langchain-community.  
Utilizar el PDF Loader para cargar el archivo.  
Especificar la ruta del archivo PDF en tu entorno de trabajo (como Google Colaboratory).  
Extraer texto y metadatos para manipularlos o realizar consultas.  
b) Carga de CSVs  
El CSV Loader en LangChain te permite cargar archivos CSV, donde cada fila se trata como un documento independiente. Este loader es especialmente útil cuando deseas analizar grandes volúmenes de datos estructurados.  

Uso del CSV Loader: Puedes cargar un archivo CSV, acceder a cada fila y columna, y realizar consultas sobre el contenido. Además, puedes combinar esta funcionalidad con técnicas de búsqueda semántica para extraer información relevante.  
Pasos para cargar un CSV:  

Utilizar el CSV Loader para cargar el archivo.  
Acceder a los datos fila por fila.  
Realizar consultas sobre las columnas y extraer información específica.  
2. Uso de Vector Stores  
Una vez que cargas un archivo (ya sea PDF, CSV u otro), LangChain te permite realizar búsquedas más sofisticadas utilizando espacios vectoriales. Un vector store almacena información en forma de vectores, lo que facilita la búsqueda por similitud. Esta técnica es útil cuando queremos buscar información relacionada de manera semántica dentro de un documento o conjunto de documentos.  
  
a) Vectorización de Documentos  
Para realizar una búsqueda semántica, necesitamos convertir el texto de los documentos en vectores. Esto se logra utilizando Embeddings, que son representaciones numéricas del texto. En este curso, hemos utilizado OpenAI Embeddings, que convierte el contenido textual en números que el sistema puede comparar.  

Proceso de vectorización:  

Cargar el documento con un loader (como el PDF Loader).  
Convertir el texto a vectores utilizando Embeddings.  
Realizar búsquedas basadas en similitudes entre los vectores.    
b) Búsqueda Semántica  
Una vez vectorizados los documentos, puedes realizar búsquedas por similitud, donde se busca la proximidad entre el texto de consulta y los documentos almacenados como vectores. Esto es extremadamente útil para encontrar respuestas relevantes en grandes volúmenes de información.

Ejemplo: Si buscas un texto que incluya una frase específica, el sistema puede encontrar el contenido más cercano en significado y devolverlo como resultado.  

3. Integración de Agentes en Búsquedas de Datos  
Uno de los aspectos más potentes de LangChain es su capacidad para integrar agentes. Un agente es una entidad que puede tomar decisiones autónomas basadas en entradas del usuario y consultar datos desde diversas fuentes.  

Uso en consultas de CSV: Los agentes pueden transformar preguntas en lenguaje natural en consultas específicas sobre un archivo CSV, simplificando el acceso a los datos. Esto es útil en escenarios donde el usuario no necesita conocer la estructura de los datos, ya que el agente se encarga de traducir la consulta.    
4. Preparación para Bases Vectoriales    
En las próximas clases, veremos con mayor profundidad cómo manejar bases vectoriales. Las bases vectoriales permiten organizar y consultar información de manera más eficiente, especialmente cuando trabajamos con grandes volúmenes de texto o datos no estructur